In [1]:
import pandas as pd
import numpy as np
import os
import ast
import matplotlib.pyplot as plt
import cv2
import glob
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from tifffile import imsave
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from PIL import Image
import random
import keras

In [2]:
data_test = np.load('C:/Users/Scott/Documents/MatrixMethodsFinal-master/input_data_test/train/images/image_2.npy')
np.unique(data_test/255)

array([0.        , 0.00392157, 0.00784314, 0.01176471, 0.01568627,
       0.01960784, 0.02352941, 0.02745098, 0.03137255, 0.03529412,
       0.03921569, 0.04313725, 0.04705882, 0.05098039, 0.05490196,
       0.05882353, 0.0627451 , 0.06666667, 0.07058824, 0.0745098 ,
       0.07843137, 0.08235294, 0.08627451, 0.09019608, 0.09411765,
       0.09803922, 0.10196078, 0.10588235, 0.10980392, 0.11372549,
       0.11764706, 0.12156863, 0.1254902 , 0.12941176, 0.13333333,
       0.1372549 , 0.14117647, 0.14509804, 0.14901961, 0.15294118,
       0.15686275, 0.16078431, 0.16470588, 0.16862745, 0.17254902,
       0.17647059, 0.18039216, 0.18431373, 0.18823529, 0.19215686,
       0.19607843, 0.2       , 0.20392157, 0.20784314, 0.21176471,
       0.21568627, 0.21960784, 0.22352941, 0.22745098, 0.23137255,
       0.23529412, 0.23921569, 0.24313725, 0.24705882, 0.25098039,
       0.25490196, 0.25882353, 0.2627451 , 0.26666667, 0.27058824,
       0.2745098 , 0.27843137, 0.28235294, 0.28627451, 0.29019

In [3]:
#shapes = []
#for im in images:
#   if im.shape != (512, 512, 9):
#        shapes.append(im)
#shapes

In [4]:
def load_image(idir, ilist, t):
    images = []
    for i, image_name in enumerate(ilist):
        if image_name.split('.')[1] == 'npy':
            image = np.load(idir+image_name)
            if t:
                image = image/255
            images.append(image)
    images = np.array(images)
    return (images)

def image_loader(idir, ilist, mdir, mlist, batch_size):
    n = len(ilist)
    
    while True:
        batch_start = 0
        batch_end = batch_size
        
        while batch_start < n:
            end = min(batch_end, n)
            x = load_image(idir, ilist[batch_start:end], True)
            y = load_image(mdir, mlist[batch_start:end], False)
            yield(x, y)
            batch_start += batch_size   
            batch_end += batch_size

In [5]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras import backend as K

In [6]:
kernel_initializer =  'he_uniform'
def unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, num_classes):
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    #n = Lambda(lambda x: x / 255)(inputs)
    n = inputs

    x1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(n)
    x1 = Dropout(0.1)(x1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(x1)
    y1 = MaxPooling2D((2, 2))(x1)
    
    x2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(y1)
    x2 = Dropout(0.1)(x2)
    x2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(x2)
    y2 = MaxPooling2D((2, 2))(x2)
     
    x3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(y2)
    x3 = Dropout(0.2)(x3)
    x3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(x3)
    y3 = MaxPooling2D((2, 2))(x3)
     
    x4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(y3)
    x4 = Dropout(0.2)(x4)
    x4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(x4)
    y4 = MaxPooling2D(pool_size=(2, 2))(x4)
     
    x5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(y4)
    x5 = Dropout(0.3)(x5)
    x5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(x5)
    y5 = MaxPooling2D(pool_size=(2, 2))(x5)
    
    x6 = Conv2D(512, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(y5)
    x6 = Dropout(0.3)(x6)
    x6 = Conv2D(512, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(x6)
    
    #Expansive path
    y7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(x6)
    y7 = concatenate([y7, x5])
    x7 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(y7)
    x7 = Dropout(0.2)(x7)
    x7 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(x7)
    
    y8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(x7)
    y8 = concatenate([y8, x4])
    x8 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(y8)
    x8 = Dropout(0.2)(x8)
    x8 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(x8)
     
    y9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(x8)
    y9 = concatenate([y9, x3])
    x9 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(y9)
    x9 = Dropout(0.2)(x9)
    x9 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(x9)
     
    y10 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(x9)
    y10 = concatenate([y10, x2])
    x10 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(y10)
    x10 = Dropout(0.1)(x10)
    x10 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(x10)
     
    y11 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(x10)
    y11 = concatenate([y11, x1], axis=3)
    x11 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(y11)
    x11 = Dropout(0.1)(x11)
    x11 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(x11)
     
    outputs = Conv2D(num_classes, (1, 1), activation='sigmoid')(x11)
     
    model = Model(inputs=[inputs], outputs=[outputs])
    
    model.summary()
    
    return model

def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)



In [7]:
train_img_dir = "C:/Users/Scott/Documents/MatrixMethodsFinal-master/input_data_test/train/images/"
train_mask_dir = "C:/Users/Scott/Documents/MatrixMethodsFinal-master/input_data_test/train/masks/"

val_img_dir = "C:/Users/Scott/Documents/MatrixMethodsFinal-master/input_data_test/val/images/"
val_mask_dir = "C:/Users/Scott/Documents/MatrixMethodsFinal-master/input_data_test/val/masks/"

train_img_list=os.listdir(train_img_dir)
train_mask_list = os.listdir(train_mask_dir)

val_img_list=os.listdir(val_img_dir)
val_mask_list = os.listdir(val_mask_dir)

In [8]:
batch_size = 8

train_img_datagen = image_loader(train_img_dir, train_img_list, 
                                train_mask_dir, train_mask_list, batch_size)

val_img_datagen = image_loader(val_img_dir, val_img_list, 
                                val_mask_dir, val_mask_list, batch_size)

In [9]:
import segmentation_models as sm
dice_loss = sm.losses.DiceLoss(class_weights=np.array([.3, .7])) 
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)
epoch = 50
metrics = [sm.metrics.IOUScore(class_weights=np.array([.3, .7]),  threshold=0.5), jacard_coef]
LR = 0.0001
optim = keras.optimizers.Adam(LR)

Segmentation Models: using `keras` framework.


In [10]:
steps_per_epoch = len(train_img_list)//batch_size
val_steps_per_epoch = len(val_img_list)//batch_size

In [11]:
model = unet_model(IMG_HEIGHT=512, 
                   IMG_WIDTH=512,  
                   IMG_CHANNELS=12, 
                   num_classes=2)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 1  0           []                               
                                2)]                                                               
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 16  1744        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 dropout (Dropout)              (None, 512, 512, 16  0           ['conv2d[0][0]']                 
                                )                                                             

                                                                                                  
 conv2d_transpose_2 (Conv2DTran  (None, 128, 128, 64  32832      ['conv2d_15[0][0]']              
 spose)                         )                                                                 
                                                                                                  
 concatenate_2 (Concatenate)    (None, 128, 128, 12  0           ['conv2d_transpose_2[0][0]',     
                                8)                                'conv2d_5[0][0]']               
                                                                                                  
 conv2d_16 (Conv2D)             (None, 128, 128, 64  73792       ['concatenate_2[0][0]']          
                                )                                                                 
                                                                                                  
 dropout_8

In [12]:
model.compile(optimizer = optim, loss=total_loss, metrics=metrics)
print(model.summary())

print(model.input_shape)
print(model.output_shape)
keras.backend.clear_session()
history=model.fit(train_img_datagen,
          steps_per_epoch=steps_per_epoch,
          epochs=epoch,
          verbose=1,
          validation_data=val_img_datagen,
          validation_steps=val_steps_per_epoch,
          )


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 1  0           []                               
                                2)]                                                               
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 16  1744        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 dropout (Dropout)              (None, 512, 512, 16  0           ['conv2d[0][0]']                 
                                )                                                             

                                                                                                  
 conv2d_transpose_2 (Conv2DTran  (None, 128, 128, 64  32832      ['conv2d_15[0][0]']              
 spose)                         )                                                                 
                                                                                                  
 concatenate_2 (Concatenate)    (None, 128, 128, 12  0           ['conv2d_transpose_2[0][0]',     
                                8)                                'conv2d_5[0][0]']               
                                                                                                  
 conv2d_16 (Conv2D)             (None, 128, 128, 64  73792       ['concatenate_2[0][0]']          
                                )                                                                 
                                                                                                  
 dropout_8

Epoch 17/50
184/184 [==============================] - 46s 250ms/step - loss: 0.8525 - iou_score: 0.1502 - jacard_coef: 0.7688 - val_loss: 0.8569 - val_iou_score: 0.1495 - val_jacard_coef: 0.9934
Epoch 18/50
184/184 [==============================] - 46s 250ms/step - loss: 0.8572 - iou_score: 0.1495 - jacard_coef: 0.9930 - val_loss: 0.8569 - val_iou_score: 0.1495 - val_jacard_coef: 0.9934
Epoch 19/50
184/184 [==============================] - 46s 251ms/step - loss: 0.8572 - iou_score: 0.1495 - jacard_coef: 0.9930 - val_loss: 0.8569 - val_iou_score: 0.1495 - val_jacard_coef: 0.9934
Epoch 20/50
184/184 [==============================] - 46s 250ms/step - loss: 0.8572 - iou_score: 0.1495 - jacard_coef: 0.9931 - val_loss: 0.8569 - val_iou_score: 0.1495 - val_jacard_coef: 0.9934
Epoch 21/50
184/184 [==============================] - 46s 251ms/step - loss: 0.8572 - iou_score: 0.1495 - jacard_coef: 0.9931 - val_loss: 0.8569 - val_iou_score: 0.1495 - val_jacard_coef: 0.9934
Epoch 22/50
184/184 

In [13]:
img, msk = train_img_datagen.__next__()

In [14]:
img.shape

(8, 512, 512, 12)

In [15]:
msk.shape

(8, 512, 512, 2)

In [17]:
model.save('ct2d37.hdf5')

In [16]:
msk

array([[[[1., 0.],
         [1., 0.],
         [1., 0.],
         ...,
         [1., 0.],
         [1., 0.],
         [1., 0.]],

        [[1., 0.],
         [1., 0.],
         [1., 0.],
         ...,
         [1., 0.],
         [1., 0.],
         [1., 0.]],

        [[1., 0.],
         [1., 0.],
         [1., 0.],
         ...,
         [1., 0.],
         [1., 0.],
         [1., 0.]],

        ...,

        [[1., 0.],
         [1., 0.],
         [1., 0.],
         ...,
         [1., 0.],
         [1., 0.],
         [1., 0.]],

        [[1., 0.],
         [1., 0.],
         [1., 0.],
         ...,
         [1., 0.],
         [1., 0.],
         [1., 0.]],

        [[1., 0.],
         [1., 0.],
         [1., 0.],
         ...,
         [1., 0.],
         [1., 0.],
         [1., 0.]]],


       [[[1., 0.],
         [1., 0.],
         [1., 0.],
         ...,
         [1., 0.],
         [1., 0.],
         [1., 0.]],

        [[1., 0.],
         [1., 0.],
         [1., 0.],
         ...,
   